# RSI_Hedge_v2.3

## Setup

### Packages

In [3]:
from utils.packages import *

### Inputs and Parameters

#### Read Yaml files

In [5]:
temp_file = 'config/access_keys.yaml'
with open(temp_file) as temp_file:
    config = yaml.load(temp_file)
    
temp_file = 'config/currencies.yaml'
with open(temp_file) as temp_file:
    currs = yaml.load(temp_file)        

#### API Setup

In [6]:
data = {}
data['instrument'] = "EUR_USD"
data['pip_size'] = currs['currs'][data['instrument']]['pip_size']

access_token = config['oanda_demo_hedge']['token']
accountID = config['oanda_demo_hedge']['account_id']
params = {'instruments': data['instrument']}


api = API(access_token = access_token)
request_stream_data = pricing.PricingStream(accountID=accountID, params=params)
response_stream = api.request(request_stream_data)

## Functions

In [ ]:
#--------------------------------------------------------------------------------------------------------------------------
def reset_data():
    global data
    #Parameters ------------------------------------------
    data['order_num'] = 1

    data['val_profit_pip'] = 0.5
    data['val_stop_loss'] = 20    
    data['val_start_hedge'] = 2

    data['rsi_len'] = 14
    data['rsi_ob'] = 70
    data['rsi_os'] = 30
    
    data['iter_limit_flag'] = False
    data['iter_max'] = 180
        
        
    #Declaration ------------------------------------------

    data['ts_date_val'] = 0
    data['ts_time_val'] = 0
    data['ts_time_fraction'] = 0
    
    data['price_ask'] = 0
    data['price_bid'] = 0
    data['price_stop'] = 0
    data['price_tick'] = 0

    data['price_order_ask'] = 0
    data['price_order_bid'] = 0
    data['price_spread'] = 0
    
    data['rsi_val'] = 0
    data['rsi_status'] = None
    
    data['long_profit_val']      = 0
    data['long_max_profit']      = 0
    data['long_buffer_profit']   = 0
    
    data['short_profit_val']      = 0
    data['short_max_profit']      = 0
    data['short_buffer_profit']   = 0    

    data['profit_val_both'] = 0
    
    data['order_create'] = False
    data['order_current_open'] = False
    data['positions_info'] = None
    data['positions_long'] = 0
    data['positions_short'] = 0

    data['response_order'] = 0
    data['response_close'] = 0

    data['list_tick'] = collections.deque([])
    data['list_up'] = collections.deque([])
    data['list_down'] = collections.deque([])
    data['list_AvgGain'] = collections.deque([])
    data['list_Avgloss'] = collections.deque([])
    data['list_RS'] = collections.deque([])
    data['list_RSI'] = collections.deque([])
    
    data['rsi_max'] = 0
    data['rsi_min'] = 100
    
    data['pip_take_profit'] = data['val_profit_pip'] * data['pip_size']    
    data['pip_stop_loss'] = data['val_stop_loss'] * data['pip_size']
    data['pip_start_hedge'] = data['val_start_hedge'] * data['pip_size']
    
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
# Get timestamp of the price and segregate it
def get_date_time(resp):
    global data
    time_stamp = resp['time']
    data['ts_date_val'], full_time = time_stamp.split(sep = 'T')
    data['ts_time_val'], data['ts_time_fraction'] = full_time.split(sep = '.')
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
# Get bid and ask prices
def get_prices(resp):    
    global data
    data['price_bid'] = float(resp['bids'][0]['price'])    
    data['price_ask'] = float(resp['asks'][0]['price'])
    data['price_spread'] = data['price_ask'] - data['price_bid']
    data['price_tick'] = (data['price_ask'] + data['price_bid']) / 2
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
def before_rsi_len():
    global data
    data['list_tick'].append(data['price_tick'])

    if len(data['list_tick']) == 1:
        data['list_up'].append(0)
        data['list_down'].append(0)
        data['list_AvgGain'].append(0)
        data['list_Avgloss'].append(0)
        data['list_RS'].append(0)
        data['list_RSI'].append(0)
    elif len(data['list_tick']) > 1:
        old_price = data['list_tick'][len(data['list_tick'])-2]
        new_price = data['price_tick']
        if new_price > old_price:
            data['list_up'].append(new_price - old_price)
            data['list_down'].append(0)
            data['list_AvgGain'].append(np.mean(data['list_up']))
            data['list_Avgloss'].append(np.mean(data['list_down']))
            data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
            data['rsi_val'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
            data['list_RSI'].append(data['rsi_val'])            
        elif old_price > new_price:
            data['list_up'].append(0)
            data['list_down'].append(old_price - new_price)
            data['list_AvgGain'].append(np.mean(data['list_up']))
            data['list_Avgloss'].append(np.mean(data['list_down']))
            data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
            data['rsi_val'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
            data['list_RSI'].append(data['rsi_val'])            
        elif old_price == new_price:
            data['list_up'].append(0)
            data['list_down'].append(0)
            data['list_AvgGain'].append(np.mean(data['list_up']))
            data['list_Avgloss'].append(np.mean(data['list_down']))
            data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
            data['rsi_val'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
            data['list_RSI'].append(data['rsi_val'])            
    display.clear_output(wait = True)
    print(f"Building RSI Tick list : {len(data['list_tick'])}")
    return()
#==========================================================================================================================    


#--------------------------------------------------------------------------------------------------------------------------
def after_rsi_len():
    global data
    data['list_up'].popleft()
    data['list_down'].popleft()
    data['list_AvgGain'].popleft()
    data['list_Avgloss'].popleft()
    data['list_RS'].popleft()
    data['list_RSI'].popleft()
    data['list_tick'].popleft()
    data['list_tick'].append(data['price_tick'])

    old_price = data['list_tick'][len(data['list_tick'])-2]
    new_price = data['price_tick']
    if new_price > old_price:
        data['list_up'].append(new_price - old_price)
        data['list_down'].append(0)
        data['list_AvgGain'].append(np.mean(data['list_up']))
        data['list_Avgloss'].append(np.mean(data['list_down']))
        data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
        data['rsi_val'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
        data['list_RSI'].append(data['rsi_val'])
    elif old_price > new_price:
        data['list_up'].append(0)
        data['list_down'].append(old_price - new_price)
        data['list_AvgGain'].append(np.mean(data['list_up']))
        data['list_Avgloss'].append(np.mean(data['list_down']))
        data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
        data['rsi_val'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
        data['list_RSI'].append(data['rsi_val'])
    elif old_price == new_price:
        data['list_up'].append(0)
        data['list_down'].append(0)
        data['list_AvgGain'].append(np.mean(data['list_up']))
        data['list_Avgloss'].append(np.mean(data['list_down']))
        data['list_RS'].append(data['list_AvgGain'][len(data['list_AvgGain'])-1]/data['list_Avgloss'][len(data['list_Avgloss'])-1])
        data['rsi_val'] = 100 - (100/(1+data['list_RS'][len(data['list_RS'])-1]))
        data['list_RSI'].append(data['rsi_val'])            

    return()
#==========================================================================================================================    


#--------------------------------------------------------------------------------------------------------------------------
def check_for_open_orders():
    global data
    
    request_position_data = positions.OpenPositions(accountID=accountID)
    data['positions_info'] = api.request(request_position_data)

    if len(data['positions_info']['positions']) == 0:
        data['order_current_open'] = False

    elif len(data['positions_info']['positions']) == 1:
        data['positions_long'] = abs(int(data['positions_info']['positions'][0]['long']['units']))
        data['positions_short'] = abs(int(data['positions_info']['positions'][0]['short']['units']))

        if data['positions_long'] >= 1 and data['positions_short'] == 0:
            data['order_current_open'] = 'long'
        elif data['positions_long'] == 0 and data['positions_short'] >= 1:
            data['order_current_open'] = 'short'                        
        elif data['positions_long'] >= 1 and data['positions_short'] >= 1:
            data['order_current_open'] = 'both'            

    return()
#==========================================================================================================================    

#--------------------------------------------------------------------------------------------------------------------------
def check_rsi_status():
    global data
    if not data['order_current_open']:
        if data['rsi_val'] <= data['rsi_os']:
            data['rsi_status'] = 'OS'
        elif data['rsi_val'] >= data['rsi_ob']:
            data['rsi_status'] = 'OB'
        else:
            data['rsi_status'] = None
            data['order_create'] = None
    return()
#==========================================================================================================================

#--------------------------------------------------------------------------------------------------------------------------
def rsi_peak_check():
    global data
    if not data['order_current_open']:
        if data['rsi_status'] == 'OB':
            data['rsi_max'] = max(data['rsi_max'], data['rsi_val'])
            if data['rsi_val'] < data['rsi_max']:
                data['order_create'] = 'short'
                data['rsi_max'] = 0
                
        if data['rsi_status'] == 'OS':
            data['rsi_min'] = min(data['rsi_min'], data['rsi_val'])
            if data['rsi_val'] > data['rsi_min']:
                data['order_create'] = 'long'
                data['rsi_min'] = 100                
#==========================================================================================================================

#--------------------------------------------------------------------------------------------------------------------------
def make_order():
    global data
    if not data['order_current_open']:
        if data['order_create'] == 'long':            
            data['order_val'] = data['order_num'] * 1
            
            data['price_stop'] = data['price_ask'] - data['pip_stop_loss']
            stopLossOnFill = StopLossDetails(price=data['price_stop'])
                       
            ordr = MarketOrderRequest(instrument = data['instrument'], 
                                      units=data['order_val'], 
                                      stopLossOnFill=stopLossOnFill.data)

            order_request_data = orders.OrderCreate(accountID, data=ordr.data)
            
            data['response_order'] = api.request(order_request_data)
            return()

        if data['order_create'] == 'short':
            data['order_val'] = data['order_num'] * -1
            
            data['price_stop'] = data['price_bid'] + data['pip_stop_loss']
            stopLossOnFill = StopLossDetails(price=data['price_stop'])

            ordr = MarketOrderRequest(instrument = data['instrument'], 
                                      units=data['order_val'], 
                                      stopLossOnFill=stopLossOnFill.data)
            
            order_request_data = orders.OrderCreate(accountID, data=ordr.data)
            
            data['response_order'] = api.request(order_request_data)
            return()
#==========================================================================================================================
          
    
#--------------------------------------------------------------------------------------------------------------------------
def take_profit():
    global data
    
    if data['order_current_open'] == 'both':
        data['price_order_ask'] = float(data['positions_info']['positions'][0]['long']['averagePrice'])
        data['long_profit_val'] = data['price_bid'] - data['price_order_ask']
        data['price_order_bid'] = float(data['positions_info']['positions'][0]['short']['averagePrice'])
        data['short_profit_val'] = data['price_order_bid'] - data['price_ask']
        data['profit_val_both'] =  data['long_profit_val'] + data['short_profit_val']
        
        if data['profit_val_both'] >= data['pip_take_profit']:
            data_long = {"longUnits": "ALL"}
            order_close_data = positions.PositionClose(accountID=accountID,
                                        instrument=data['instrument'],
                                        data=data_long)
            data['response_close'] = api.request(order_close_data)
            
            data_short = {"shortUnits": "ALL"}
            order_close_data = positions.PositionClose(accountID=accountID,
                                        instrument=data['instrument'],
                                        data=data_short)
            data['response_close'] = api.request(order_close_data)    
            
            check_for_open_orders()
            if not data['order_current_open']:
                reset_data()
    
    if data['order_current_open'] == 'long' or data['order_current_open'] == 'both':
        data['price_order_ask'] = float(data['positions_info']['positions'][0]['long']['averagePrice'])
        data['long_profit_val']      = data['price_bid'] - data['price_order_ask']
        data['long_max_profit']      = max(data['long_max_profit'], data['long_profit_val'])
        data['long_buffer_profit']   = data['long_max_profit'] - data['pip_take_profit']
        data['long_buffer_profit']   = max(data['long_buffer_profit'], data['pip_take_profit'])
        if data['long_buffer_profit'] >= data['long_profit_val'] >= data['pip_take_profit']:
            data_long = {"longUnits": "ALL"}
            order_close_data = positions.PositionClose(accountID=accountID,
                                        instrument=data['instrument'],
                                        data=data_long)
            data['response_close'] = api.request(order_close_data)
            
            check_for_open_orders()
            if not data['order_current_open']:
                reset_data()
                
    if data['order_current_open'] == 'short' or data['order_current_open'] == 'both':
        data['price_order_bid'] = float(data['positions_info']['positions'][0]['short']['averagePrice'])
        data['short_profit_val']      = data['price_order_bid'] - data['price_ask']
        data['short_max_profit']      = max(data['short_max_profit'], data['short_profit_val'])
        data['short_buffer_profit']   = data['short_max_profit'] - data['pip_take_profit']
        data['short_buffer_profit']   = max(data['short_buffer_profit'], data['pip_take_profit'])
        if data['short_buffer_profit'] >= data['short_profit_val'] >= data['pip_take_profit']:
            data_short = {"shortUnits": "ALL"}
            order_close_data = positions.PositionClose(accountID=accountID,
                                        instrument=data['instrument'],
                                        data=data_short)
            data['response_close'] = api.request(order_close_data)  
            
            check_for_open_orders()
            if not data['order_current_open']:
                reset_data()
    return()
#==========================================================================================================================
          
    
#--------------------------------------------------------------------------------------------------------------------------
def make_hedge():
    if data['order_current_open'] == 'long':
        data['price_order_ask'] = float(data['positions_info']['positions'][0]['long']['averagePrice'])
        data['long_profit_val']      = data['price_bid'] - data['price_order_ask']
        if data['long_profit_val'] <= -data['pip_start_hedge']:
            data['order_val'] = data['order_num'] * -1
            data['price_stop'] = data['price_bid'] + data['pip_stop_loss']
            stopLossOnFill = StopLossDetails(price=data['price_stop'])
            ordr = MarketOrderRequest(instrument = data['instrument'], 
                                      units=data['order_val'], 
                                      stopLossOnFill=stopLossOnFill.data)
            order_request_data = orders.OrderCreate(accountID, data=ordr.data)
            data['response_order'] = api.request(order_request_data)
            
    if data['order_current_open'] == 'short':
        data['price_order_bid'] = float(data['positions_info']['positions'][0]['short']['averagePrice'])
        data['short_profit_val']      = data['price_order_bid'] - data['price_ask']
        if data['short_profit_val'] <= -data['pip_start_hedge']:
            data['order_val'] = data['order_num'] * 1            
            data['price_stop'] = data['price_ask'] - data['pip_stop_loss']
            stopLossOnFill = StopLossDetails(price=data['price_stop'])                       
            ordr = MarketOrderRequest(instrument = data['instrument'], 
                                      units=data['order_val'], 
                                      stopLossOnFill=stopLossOnFill.data)
            order_request_data = orders.OrderCreate(accountID, data=ordr.data)            
            data['response_order'] = api.request(order_request_data)        
    return()
#==========================================================================================================================


#--------------------------------------------------------------------------------------------------------------------------
def dict_to_df():
    global data
    data_rec = pd.DataFrame()    
    ordered_col_list = ['ts_date_val', 'ts_time_val', 'ts_time_fraction', 'instrument', 'pip_size', 'order_num', 'val_profit_pip', 'val_stop_loss', 'val_start_hedge', 'rsi_len', 'rsi_ob', 'rsi_os', 'iter_limit_flag', 'iter_max', 'rsi_status', 'rsi_val', 'order_create', 'order_current_open', 'price_tick', 'price_ask', 'price_bid', 'price_order_ask', 'price_order_bid', 'price_stop', 'positions_long', 'long_buffer_profit', 'long_max_profit', 'long_profit_val', 'positions_short', 'short_buffer_profit', 'short_max_profit', 'short_profit_val', 'profit_val_both', 'pip_take_profit', 'pip_stop_loss', 'pip_start_hedge', 'rsi_max', 'rsi_min', 'price_spread', 'positions_info', 'response_order', 'response_close', 'list_tick', 'list_up', 'list_down', 'list_AvgGain', 'list_Avgloss', 'list_RS', 'list_RSI']

    data_rec = data_rec.append(data, ignore_index=True)
    data_rec = data_rec[ordered_col_list]
    
    return(data_rec)
#==========================================================================================================================

## Core Engine

In [ ]:
reset_data()

for i, resp in enumerate(response_stream): 
    
    #Iteration Control ------------------------------------------
    if data['iter_limit_flag'] and i >= data['iter_max']:
        break
    if resp['type'] == 'HEARTBEAT': # Heart beat response to keep the api connection alive (Avoid timeout)
        continue
    
    
    # Date and Data ---------------------------------------------
    get_date_time(resp)
    get_prices(resp)

    
    # Build RSI ---------------------------------------------
    if len(data['list_tick']) < data['rsi_len']:
        before_rsi_len()          
        continue
    elif len(data['list_tick']) == data['rsi_len']:
        after_rsi_len()        
    
    check_for_open_orders()    
    check_rsi_status()
    rsi_peak_check()

    
    # Orders and Hedges -------------------------------------------
    make_order()
    check_for_open_orders()
    make_hedge()
    check_for_open_orders()      
    
    
    # Take Profit ---------------------------------------------
    take_profit()
    check_for_open_orders()   
                
    # Reporting ---------------------------------------------
    data_rec = dict_to_df()
    print(data_rec.head().transpose())    
    display.clear_output(wait = True)

In [ ]:
data['positions_info']

In [ ]:
data['ordered_units'] = int(data['response_order']['orderFillTransaction']['units'])
data['ordered_price'] = float(data['response_order']['orderFillTransaction']['price'])
data['ordered_pl'] = float(data['response_order']['orderFillTransaction']['pl'])

In [7]:
import oandapyV20.endpoints.transactions as trans

In [8]:
%%time
id_num = 1233
r = trans.TransactionsSinceID(accountID=accountID, params={"id": id_num})
trans_repsonse = api.request(r)

Wall time: 42.9 s


In [27]:
for i, val in enumerate(trans_response["transactions"]):
    trans_type = val["type"]
    print('-----------------------------')
    if trans_type == 'ORDER_FILL':
        order_reason = val["reason"]
        if order_reason == 'MARKET_ORDER':
            print(f'order_reason : {order_reason}')
            print(f'tradeID : {val["tradeOpened"]["tradeID"]}')
            print(f'units : {val["tradeOpened"]["units"]}')
            print(f'price : {val["tradeOpened"]["price"]}')
            print(f'realizedPL : 0')
        elif order_reason == 'MARKET_ORDER_TRADE_CLOSE':
            print(f'order_reason : {order_reason}')
            print(f'tradeID : {val["tradesClosed"][0]["tradeID"]}')
            print(f'units : {val["tradesClosed"][0]["units"]}')
            print(f'price : {val["tradesClosed"][0]["price"]}')
            print(f'realizedPL : {val["tradesClosed"][0]["realizedPL"]}')
        elif order_reason == 'MARKET_ORDER_POSITION_CLOSEOUT':
            print(f'order_reason : {order_reason}')
            print(f'tradeID : {val["tradesClosed"][0]["tradeID"]}')
            print(f'units : {val["tradesClosed"][0]["units"]}')
            print(f'price : {val["tradesClosed"][0]["price"]}')
            print(f'realizedPL : {val["tradesClosed"][0]["realizedPL"]}')

-----------------------------
order_reason : MARKET_ORDER
tradeID : 1234
units : -1
price : 1.18394
realizedPL : 0
-----------------------------
-----------------------------
-----------------------------
order_reason : MARKET_ORDER
tradeID : 1237
units : 1
price : 1.18415
realizedPL : 0
-----------------------------
-----------------------------
-----------------------------
order_reason : MARKET_ORDER_TRADE_CLOSE
tradeID : 1234
units : 1
price : 1.18425
realizedPL : -0.0004
-----------------------------
-----------------------------
-----------------------------
order_reason : MARKET_ORDER_TRADE_CLOSE
tradeID : 1237
units : -1
price : 1.18416
realizedPL : 0.0000
-----------------------------
-----------------------------
-----------------------------
order_reason : MARKET_ORDER
tradeID : 1246
units : -1
price : 1.18453
realizedPL : 0
-----------------------------
-----------------------------
-----------------------------
order_reason : MARKET_ORDER
tradeID : 1249
units : 1
price : 1